# Курс Специализация Data Science Дипломный проект. Бриф учебного кейса 
## «Модель прогнозирования стоимости жилья для агентства недвижимости»

В данном ноутбуке мы сделаем следующее:
* Подгрузим обученную модель CatBoostRegressor
* Сохраним названия колонок в отдельных файлах
* Напишем функцию для предсказания стоимости недвижимости по входным параметрам
* Сохраним наиболее привлекательные варианты в json-файл

In [21]:
import numpy as np 
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings("ignore")

In [22]:
# Загрузка сохраненной модели из файла pickle
with open("prototype/server/artifacts/best_cb_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

In [23]:
# создадим тестовый набор 
data = [
    ('Active', 3.0, 'Washington', 801, 20004, 'DC', False, 'other', 1991, False, False, False, False, 6.0, 0.2),
    ('Active', 2.0, 'Dallas', 832, 75219, 'TX', False, 'condo', 1998, False, True, False, False, 2.6, 0.6),
    ('Active', 4.0, 'Dallas', 2102, 75219, 'TX', False, 'condo', 2019, False, True, False, False, 2.0, 0.1),
    ('Active', 2.0, 'Dallas', 837, 75219, 'TX', False, 'condo', 1998, False, True, False, False, 2.6, 0.6),
    ('Active', 2.0, 'Dallas', 2785, 75219, 'TX', False, 'condo', 2015, False, True, False, True, 2.6, 0.7)
]

columns = ['status', 'baths', 'city', 'sqft', 'zipcode', 'state', 'pool_encoded', 'Type', 'Year built', 'Heating_encoded', 'Cooling_encoded', 'Parking_encoded', 'fireplace_encoded', 'school_rating _mean', 'school_dist_min']

df_test = pd.DataFrame(data, columns=columns)

df_test.head()

,status,baths,city,sqft,zipcode,state,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,3.0,Washington,801,20004,DC,False,other,1991,False,False,False,False,6.0,0.2
1,Active,2.0,Dallas,832,75219,TX,False,condo,1998,False,True,False,False,2.6,0.6
2,Active,4.0,Dallas,2102,75219,TX,False,condo,2019,False,True,False,False,2.0,0.1
3,Active,2.0,Dallas,837,75219,TX,False,condo,1998,False,True,False,False,2.6,0.6
4,Active,2.0,Dallas,2785,75219,TX,False,condo,2015,False,True,False,True,2.6,0.7


In [24]:
def preprocess_and_predict(df_input, model):
    '''Pre-processes input data and makes predictions using the provided model.'''
    
    def log_data(df_input):
        df_output = df_input.copy()
        df_output['zipcode'] = df_output['zipcode'].astype(str)
        df_output['Year built'] = df_output['Year built'].astype(str)
        
        #scaler = MinMaxScaler()
        for column in ['baths', 'sqft', 'school_rating _mean', 'school_dist_min']:
            #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
            df_output[column] = df_output[column].apply(lambda x: abs(x))
            constant = 1e-6
            df_output[column] = np.log(df_output[column] + constant)
        return df_output

    X_test = log_data(df_input)
    y_test_pred_loaded = model.predict(X_test)
    target = np.exp(y_test_pred_loaded)
    rounded_target = np.round(target)
    print(rounded_target)
    
    return rounded_target

In [25]:
predictions = preprocess_and_predict(df_test, loaded_model)


[506389. 224225. 551250. 224081. 626743.]


In [26]:
# сохраним названия колонок
import json
columns = {
    'data_columns' : [col.lower() for col in df_test.columns]
}
with open("prototype/server/artifacts/columns.json","w") as f:
    f.write(json.dumps(columns))

In [27]:
#сохраним тестовый датасет
cheap_flats = df_test.to_json(orient="index")
with open("prototype/server/artifacts/cheap_flats.json","w") as f:
    f.write(json.dumps(cheap_flats))